# Nome: Matheus da Silva Oliveira
# DRE: 118178020

## 1

Será usada a [biblioteca Test](https://docs.julialang.org/en/v1/stdlib/Test/) para facilitar nos units tests.

In [14]:
using LinearAlgebra
using Test

### RESOLVE_DIAGONAL

Como sabemos, quando a matriz $A$ é diagonal, o sistema $Ax=b$ é determinado, pois todos os $x_i$ são determinados por apenas uma divisão, desde que a diagonal de $A$ seja não nula. Caso fosse, deveriámos olhar para o vetor $b$, mas não precisamos nos preocupar com este caso agora. Sendo assim, determinamos todos os elementos do vetor $x$ através da fórmula
$$
x_i = \frac{b_i}{a_{ii}}
$$

In [15]:
"""
    resolve_diagonal(A,b)
    
Retorna a matriz coluna x que é o resultado do sistema linear diagonal Ax=b.
A é uma matriz nxn e b nx1.
"""
function resolve_diagonal(A, b)
    # Computa n, que será o número de linhas da matriz x
    n = length(b)
    x = zeros(n)
    # Computa o vetor x
    for i=1:n
        x[i] = b[i] / A[i,i]
    end
    return x
end

resolve_diagonal

#### Testes

As matrizes para os testes são geradas randomicamente. A matriz identidade $I$ concedida pela biblioteca *LinearAlgebra* foi usada para gerar as matrizes diagonais.

In [16]:
# n é o número de linhas e colunas da matriz A_i e o número de linhas da matriz b_i
n = 5
A1 = randn()*Matrix{Float64}(I,n,n)
b1 = randn(n)

n = 6
A2 = randn()*Matrix{Float64}(I,n,n)
b2 = randn(n)

n = 2
A3 = randn()*Matrix{Float64}(I,n,n)
b3 = randn(n)

2-element Vector{Float64}:
  0.6992704287885547
 -1.5709143239404884

In [17]:
x1 = resolve_diagonal(A1,b1)
@test A1*x1 ≈ b1 

Test Passed

In [18]:
x2 = resolve_diagonal(A2,b2)
@test A2*x2 ≈ b2 

Test Passed

In [19]:
x3 = resolve_diagonal(A3,b3)
@test A3*x3 ≈ b3 

Test Passed

### RESOLVE_TRIANGULAR_SUPERIOR

Como visto nos vídeos do Abel, a fórmula para cada $x_i$ é dado por 
$$
x_i = \frac{b_i-\sum_{j=i+1}^{n} a_{ij}x_j}{a_{ii}}
$$
onde $a_{ij}$ são os elementos da matriz $A$.

In [20]:
"""
    resolve_triangular_superior(A,b)
    
Retorna a matriz coluna x que é o resultado do sistema linear Ax=b, onde
A é uma matriz nxn triangular superior e b possui tamanho nx1.
"""
function resolve_triangular_superior(A, b)
    # Computa n, que será o número de linhas da matriz x
    n = length(b)
    x = zeros(n)
    #= Para cada linha i de A e b, começando na última linha e terminando na primeira,
    computamos cada variável nas colunas por substituição direta=# 
    for i = n:-1:1
        # x_i = b_i no começo de cada iteração
        x[i] = b[i]
        for j = i+1:n
            # Subtraímos de b_i o somatório da fórmula
            x[i] = x[i] - (A[i,j]*x[j])
        end
        # Dividimos por a_ii 
        x[i] = x[i]/A[i,i]
    end
     # Retorna a matriz coluna resultado
    return x
end

resolve_triangular_superior

#### Testes

Como o $det(A) \neq 0$, sendo A uma matriz triangular superior ou inferior, então o sistema é determinado, visto que o determinante de uma matriz triangular superior ou inferior é o produto de sua diagonal principal. Sendo assim, utilizamos a função [UpperTriangular](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.UpperTriangular) para gerar as matrizes $A_i$. 

In [21]:
n = 5
A1 = randn(n,n)
A1 = UpperTriangular(A1)
b1 = randn(n)

n = 6
A2 = randn(n,n)
A2 = UpperTriangular(A2)
b2 = randn(n)

n = 2
A3 = randn(n,n)
A3 = UpperTriangular(A3)
b3 = randn(n)

2-element Vector{Float64}:
 -1.9588415265107613
 -1.546812113163189

In [22]:
x1 = resolve_triangular_superior(A1,b1)
@test A1*x1 ≈ b1 

Test Passed

In [23]:
x2 = resolve_triangular_superior(A2,b2)
@test A2*x2 ≈ b2 

Test Passed

In [24]:
x3 = resolve_triangular_superior(A3,b3)
@test A3*x3 ≈ b3 

Test Passed

### RESOLVE_TRIANGULAR_INFERIOR

Como visto nos vídeos do Abel, a fórmula para cada $x_i$ é dado por 
$$
x_i = \frac{b_i-\sum_{j=1}^{i-1} a_{ij}x_j}{a_{ii}}
$$

In [25]:
"""
    resolve_triangular_inferior(A,b)
    
Retorna a matriz coluna x que é o resultado do sistema linear Ax=b, onde
A é uma matriz nxn triangular inferior e b possui tamanho nx1.
"""
function resolve_triangular_inferior(A, b)
    # Computa n, que será o número de linhas da matriz x
    n = length(b)
    x = zeros(n)
    #= Para cada linha i de A e b, começando na primeira linha e terminando na última,
    computamos cada variável nas colunas por substituição direta=# 
    for i = 1:n
        # x_i = b_i no começo de cada iteração
        x[i] = b[i]
        for j = 1:(i - 1)
            # Subtraímos de b_i o somatório da fórmula
            x[i] = x[i] - (A[i,j]*x[j])
        end
        # Dividimos por a_ii 
        x[i] = x[i]/A[i,i]
    end
    # Retorna a matriz coluna resultado
    return x
end

resolve_triangular_inferior

#### Testes

O resultado visto na parte da matriz triangular superior pode ser utilizado aqui para geração dos testes. Dessa forma, seguimos o mesmo algoritmo, mas agora utilizamos a função [LowerTriangular](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.LowerTriangular) para gerar matrizes triangulares inferiores.

In [26]:
n = 5
A1 = randn(n,n)
A1 = LowerTriangular(A1)
b1 = randn(n)

n = 6
A2 = randn(n,n)
A2 = LowerTriangular(A2)
b2 = randn(n)

n = 2
A3 = randn(n,n)
A3 = LowerTriangular(A3)
b3 = randn(n)

2-element Vector{Float64}:
  1.4498742775076336
 -0.1904348119292122

In [27]:
x1 = resolve_triangular_inferior(A1,b1)
@test A1*x1 ≈ b1 

Test Passed

In [28]:
x2 = resolve_triangular_inferior(A2,b2)
@test A2*x2 ≈ b2 

Test Passed

In [29]:
x3 = resolve_triangular_inferior(A3,b3)
@test A3*x3 ≈ b3 

Test Passed

### DECOMPOSICAO_LU

In [30]:
"""
    decomposicao_LU(A)
    
Retorna as matrizes L,U nxn que correspondem a igualdade L*U = A.
A é uma matriz nxn.
"""
function decomposicao_LU(A)
    A = 1.0*A
    # Pega o número de linhas de A
    n, = size(A)
    # Gera a matriz que será L 
    L = Matrix{Float64}(I,n,n)
    # Fazemos uma cópia de A para não alterar a entrada. Essa matriz será nossa matriz triangular superior.
    U = copy(A)
    # Inicializamos a constante que multiplica as linhas
    v = 0
    # A variável i representa a linha do pivo atual
    for i = 1:(n-1)
        pivo = U[i,i]
        # Para cada linha abaixo da linha do pivo, realizamos o escalonamento
        for j = (i+1):n 
            # Constante que multiplicará toda uma linha
            v = U[j,i]/pivo
            # Multiplica todos elementos da linha correspondente para balancear o sistema
            for k = 1:n  
                U[j,k] = U[j,k] - U[i,k]*v
            end
            # Zeramos os elementos manualmente devido ao erro de precisão da máquina
            U[j,i] = 0 
            # Gera a matriz triangular inferior elemento por elemento
            L[j,i] = v
        end
    end
    # Retorna a tupla (L,U)
    return L,U
end         

decomposicao_LU

#### Testes

In [31]:
A1 = [3.0 2.0 4.0; 
    1.0 1.0 2.0; 
    4.0 3.0 -2.0]

A2 = [1.0 2.0 3.0; 
    4.0 -2.0 6.0; 
    1.0 4.0 5.0]

A3 = [8.0 -4.0 -2.0; 
    -4.0 10.0 -2.0; 
    -2.0 -2.0 10.0]

3×3 Matrix{Float64}:
  8.0  -4.0  -2.0
 -4.0  10.0  -2.0
 -2.0  -2.0  10.0

In [32]:
L,U = decomposicao_LU(A1)
@test L*U ≈ A1

Test Passed

In [33]:
L,U = decomposicao_LU(A2)
@test L*U ≈ A2

Test Passed

In [34]:
L,U = decomposicao_LU(A3)
@test L*U ≈ A3

Test Passed

### INVERSA

Para encontrar a matriz inversa, é preciso resolver o sistema $AA^{-1} = I$, onde $A^{-1}$ será determinado. Contudo, podemos pensar em resolver os sistemas $AA^{-1}_j = I_j$ onde $A^{-1}_j$ e $I_j$ correspondem as colunas $j$ da inversa de $A$ e da matriz identidade. Fazendo isso, podemos determinar completamente $A^{-1}$.

In [35]:
"""
   inversa(A)
    
Retorna a inversa A^-1 da matriz A
A é uma matriz nxn.
"""
function inversa(A)
    # Calcula o tamanho da inversa
    n, = size(A)
    # Inicializa A^-1
    A_inv = zeros(n, n)
    # Gera a matriz identidade 
    Ident = Matrix{Float64}(I,n,n)
    # Computa A*A^(-1)_j = I_j para cada coluna j 
    L,U = decomposicao_LU(A)
    for j=1:n
         Y = resolve_triangular_inferior(L,Ident[:,j])
         A_inv[:,j] = resolve_triangular_superior(U,Y)
    end
    # Retorna a inversa
    return A_inv
end      

inversa

#### Testes

Para gerar os testes, precisamos que cada matriz $A$ gerada possua $det(A) \neq 0$, pois matrizes não invertíveis possuem determinante igual a 0. Com isso, o algoritmo computa uma matriz A aleatoriamente. Se $det(A) = 0$, calculamos uma nova matriz A.

In [36]:
n = 5
A = randn(n,n)
while(det(A) == 0)
    A = randn(n,n)
end
A_inv = inversa(A)
Ident = Matrix{Float64}(I,n,n)
# Testa se A*A^-1 ≈ I
@test A*A_inv ≈ Ident

Test Passed

In [37]:
n = 10
A = randn(n,n)
while(det(A) == 0)
    A = randn(n,n)
end
A_inv = inversa(A)
Ident = Matrix{Float64}(I,n,n)
# Testa se A*A^-1 ≈ I
@test A*A_inv ≈ Ident

Test Passed

In [38]:
n = 27
A = randn(n,n)
while(det(A) == 0)
    A = randn(n,n)
end
A_inv = inversa(A)
Ident = Matrix{Float64}(I,n,n)
# Testa se A*A^-1 ≈ I
@test A*A_inv ≈ Ident

Test Passed

#### Complexidade

Primeiro, ocorre a decomposição LU que possui complexidade $O(n^3)$. Para cada coluna $A^{-1}_j$ e $I_j$, chamamos as funções *resolve_triangular_superior* e *resolve_triangular_inferior*, então temos $n$ operações. Ambas as funções anteriores possuem complexidade $O(n^2)$, então temos uma complexidade $O(n^3)$ juntando com as $n$ operações para cada coluna. Somando as duas ordem, podemos dizer que a função inversa implementada possui complexidade $O(n^3)$.

*Refs.:*
- https://www.youtube.com/watch?v=5RWstMXFohQ
- https://en.wikipedia.org/wiki/Invertible_matrix

## 2

A função *resolve* soluciona o sistema $Ax=b$ através de decomposição LU, onde A é uma matriz $nxn$. Isso pode ser feito através dos passos
\begin{align*}
Ax&=b\\
LUx&=b\\
LY&=b &\text{onde } Y &= Ux\\
UX &= Y
\end{align*}
Dessa forma $X$ é o vetor coluna que procuramos.

In [39]:
"""
    resolve(A,b)
    
Retorna a matriz coluna x que é o resultado do sistema linear Ax=b, onde
A é uma matriz nxn e b possui tamanho nx1.
"""
function resolve(A,b)
    L,U = decomposicao_LU(A)
    Y = resolve_triangular_inferior(L,b)
    return resolve_triangular_superior(U,Y)
end

resolve

#### Testes

In [40]:
n = 4
A1 = randn(n,n)
b1 = A1 * ones(n)

n = 5
A2 = randn(n,n)
b2 = A2 * ones(n)

n = 10
A3 = randn(n,n)
b3 = A3 * ones(n)

10-element Vector{Float64}:
 -8.963762705051312
 -2.8602536606524605
 -3.2504715212493487
 -2.200063326470323
  0.6756367633126077
  0.4882965439581105
 -0.2934420778363517
  3.409383568714712
 -9.160634435560466
 -5.697975902170411

In [41]:
x1 = resolve(A1,b1)
@test A1*x1 ≈ b1

Test Passed

In [42]:
x2 = resolve(A2,b2)
@test A2*x2 ≈ b2

Test Passed

In [43]:
x3 = resolve(A3,b3)
@test A3*x3 ≈ b3

Test Passed

A modelagem é parecida com a vista na aula 19, a diferença é que $y''(x)$ é determinado por uma função e temos 7 pontos pois temos 6 intervalos.

In [98]:
# h = (x_1 - x_7)/n
h = 10/6
y_linha(x) = 4*(x-1)*h
y1 = 5.0
y7 = 20.0

A = [
    -2 1 0 0 0;
    1 -2 1 0 0;
    0 1 -2 1 0;
    0 0 1 -2 1;
    0 0 0 1 -2;    
]

b = [
    2*h^2*y_linha(2) - y1;
    2*h^2*y_linha(3);
    2*h^2*y_linha(4);
    2*h^2*y_linha(5);
    2*h^2*y_linha(6) - y7;
]

5-element Vector{Float64}:
  32.037037037037045
  74.07407407407409
 111.11111111111113
 148.14814814814818
 165.18518518518522

Como $A$ possui tamanho $nxn$, podemos utilizar a função *resolve*.

In [99]:
Y = resolve(A,b)

5-element Vector{Float64}:
 -208.5493827160494
 -385.0617283950617
 -487.50000000000006
 -478.82716049382725
 -322.0061728395063

In [100]:
# Adicionamos y1 e y7 em suas posições corretas na matriz Y
Y = append!([y1],Y,[y7])
# Montamos a matriz de Vandermonde. Vamos fazer interpolação polinomial de grau 2
linhas = 7
A = zeros(linhas,3)
for i=1:linhas
    x = (i-1)*h
    A[i,1] = 1
    A[i,2] = x
    A[i,3] = x^2
end
# Resolvemos o sistema e montamos a função quadrática.
c = A\Y
f(x) = c[3]*x^2 + c[2]*x + c[1]
f(3.2345)

-414.73066722222205

## 3

In [45]:
A = [1 -1/4 -1/4 0; -1/4 1 0 -1/4; -1/4 0 1 -1/4; 0 -1/4 -1/4 1]

4×4 Matrix{Float64}:
  1.0   -0.25  -0.25   0.0
 -0.25   1.0    0.0   -0.25
 -0.25   0.0    1.0   -0.25
  0.0   -0.25  -0.25   1.0

In [46]:
b = [5 50/4 15/4 45/4]

1×4 Matrix{Float64}:
 5.0  12.5  3.75  11.25

In [47]:
# A*x = b, onde A = L*U
# L*y = b, onde obtemos y para achar x
# U*x = y, obtendo x
@time begin
    L, U = decomposicao_LU(A)
    y = resolve_triangular_inferior(L, b)
    x = resolve_triangular_superior(U, y)
end

  0.045633 seconds (21.56 k allocations: 1.282 MiB, 99.87% compilation time)


4-element Vector{Float64}:
 13.125
 20.625
 11.875
 19.375

In [48]:
A = [1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
     -1/4 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
     0 -1/4 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
     0 0 -1/4 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
     0 0 0 -1/4 1 0 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
     -1/4 0 0 0 0 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
     0 -1/4 0 0 0 -1/4 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0 0 0 0;
     0 0 -1/4 0 0 0 -1/4 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0 0 0;
     0 0 0 -1/4 0 0 0 -1/4 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0 0;
     0 0 0 0 -1/4 0 0 0 -1/4 1 0 0 0 0 -1/4 0 0 0 0 0 0 0 0 0 0;
     0 0 0 0 0 -1/4 0 0 0 0 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0 0;
     0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0 0;
     0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0 0;
     0 0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 -1/4 0 0 0 -1/4 0 0 0 0 0 0;
     0 0 0 0 0 0 0 0 0 -1/4 0 0 -1/4 1 0 0 0 0 0 -1/4 0 0 0 0 0;
     0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 0 1 -1/4 0 0 0 -1/4 0 0 0 0;
     0 0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 -1/4 0 0 0 -1/4 0 0 0;
     0 0 0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 -1/4 0 0 0 -1/4 0 0;
     0 0 0 0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 -1/4 0 0 0 -1/4 0;
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 0 0 0 0 -1/4;
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 0 1 -1/4 0 0 0;
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 -1/4 0 0;
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 -1/4 0;
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1 -1/4;
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1/4 0 0 0 -1/4 1;]

25×25 Matrix{Float64}:
  1.0   -0.25   0.0    0.0    0.0   …   0.0    0.0    0.0    0.0    0.0
 -0.25   1.0   -0.25   0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0   -0.25   1.0   -0.25   0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0   -0.25   1.0   -0.25      0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0   -0.25   1.0       0.0    0.0    0.0    0.0    0.0
 -0.25   0.0    0.0    0.0    0.0   …   0.0    0.0    0.0    0.0    0.0
  0.0   -0.25   0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0   -0.25   0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0   -0.25   0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0   -0.25      0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0   …   0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0

In [49]:
b = [45/4 ; 
    20/4; 
    20/4 ; 
    20/4; 
    40/4; 
    25/4; 
    0; 
    0; 
    0; 
    20/4; 
    25/4; 
    0; 
    0; 
    0; 
    20/4; 
    25/4; 
    0; 
    0; 
    0; 
    20/4; 
    55/4; 
    30/4; 
    30/4; 
    30/4; 
    50/4]

25-element Vector{Float64}:
 11.25
  5.0
  5.0
  5.0
 10.0
  6.25
  0.0
  0.0
  0.0
  5.0
  6.25
  0.0
  0.0
  0.0
  5.0
  6.25
  0.0
  0.0
  0.0
  5.0
 13.75
  7.5
  7.5
  7.5
 12.5

In [50]:
# A*x = b, onde A = L*U
# L*y = b, onde obtemos y para achar x
# U*x = y, obtendo x
L, U = decomposicao_LU(A)
y = resolve_triangular_inferior(L, b)
x = resolve_triangular_superior(U, y)

25-element Vector{Float64}:
 22.544910929891554
 21.518050214897478
 20.87762318174459
 20.320043856439405
 19.899858605080322
 23.661593504668744
 22.64966674795377
 21.672398655641462
 20.502693638932705
 19.27939056388188
 24.45179634082966
 23.74662461660739
 22.659611053934803
 20.738941479768073
 16.715010011514508
 25.398967242042485
 25.225424323711348
 24.48047946372228
 23.078451214690272
 21.016764301255613
 26.918648303628935
 27.275625972473247
 26.958431262552672
 26.07761961401516
 24.27359597881769

In [51]:
#item 3)d)

function resolve_lago(n_linhas, margem_sup, margem_dir, margem_inf, margem_esq)
    #declarando a matriz e o vetor (Ax = b)
    A = zeros((n_linhas)^2,(n_linhas)^2)
    b = zeros(n_linhas^2,1)
    
    for i=1:(n_linhas)^2
        A[i,i] = 1   #preenchendo a diagonal
        
        if((i+1)%n_linhas == 1) #se isso acontecer, estamos olhando para a margem direita,
                                #afinal só há n_linha pontos em cada linha
            b[i] += margem_dir/4  #então esse valor é uma constante, que fica do outro lado da equação
        else 
            A[i,i+1] = -1/4     #se o ponto estiver "no meio" do quadrado, colocamos o -1/4
                                # na matriz, já que está multiplicando uma incógnita
        end
        
        
        if((i-1)%n_linhas == 0)   #se isso acontecer, estamos na margem esquerda
            b[i] += margem_esq/4
        
        else 
            A[i,i-1] = -1/4
        end
        
        if((i+n_linhas)> n_linhas^2) #nesse caso, estamos nos ultimos pontos do quadrado, 
            b[i] += margem_inf/4     #na margem inferior
        
        else 
            A[i,i+n_linhas] = -1/4
        end
        
        if((i-n_linhas) < 1)
            b[i] += margem_sup/4    #nesse caso, estamos nos primeiros pontos do quadrado, na
                                    #margem superior
        else 
            A[i,i-n_linhas] = -1/4
        end        
        
    end
    
    #resolvendo o sistema resultante
    L, U = decomposicao_LU(A)
    y = resolve_triangular_inferior(L, b)
    x = resolve_triangular_superior(U, y)
    return x
end

resolve_lago (generic function with 1 method)

In [52]:
x = resolve_lago(52,15,35,10,5) #52 linhas, ou seja, 2704 pontos (52^2), já leva 2 minutos
length(x) #só pro output não ser um vetor de milhares de linhas 

2704

## 4

Para montar o sistema, podemos pensar que todos os canos conservam o volume de água que passa por eles, em outras palavras, todo volume de água que entra em um cano deve sair dele. Com isso em mente, podemos determinar o sistema
\begin{align*}
 x_1-2000-5000 &= 0 & \text{A} \\ 
x_2-1500-2000 &= 0 & \text{B}\\
x_3-8000-1000 &= 0 & \text{C}\\
-x_2-x_4+x_6 &= 0 & \text{F}\\
-x_5+x_7-3000 &= 0 & \text{G}\\
-x_6-x_7+x_8-500&= 0 & \text{H}\\
-x_1+x_4-30000 &= 0 & \text{D}\\
-x_3+x_5-3000 &= 0 & \text{E}\\
\end{align*}
Colocando todas as constantes do lado direito das equações e montando o sistema $Ax=c$, chegamos ao resultado

In [53]:
A = [
    1 0 0 0 0 0 0 0;
    0 1 0 0 0 0 0 0;
    0 0 1 0 0 0 0 0;
    0 -1 0 -1 0 1 0 0;
    0 0 0 0 -1 0 1 0;
    0 0 0 0 0 -1 -1 1;
    -1 0 0 1 0 0 0 0;
    0 0 -1 0 1 0 0 0;
   ]

c = [
    2000+5000;
    1500+2000;
    8000+1000;
    0;
    3000;
    500; 
    30000;
    3000;  
]

resolve(A,c)

8-element Vector{Float64}:
  7000.0
  3500.0
  9000.0
 37000.0
 12000.0
 40500.0
 15000.0
 56000.0

Agora o sistema com a variável $x_9$ será
\begin{align*}
 x_1-2000-5000 &= 0 & \text{A} \\ 
x_2-1500-2000 &= 0 & \text{B}\\
x_3-8000-1000 &= 0 & \text{C}\\
-x_2-x_4+x_6 &= 0 & \text{F}\\
-x_5+x_7-3000 &= 0 & \text{G}\\
-x_6-x_7+x_8-500&= 0 & \text{H}\\
-x_1+x_4-30000 -x_9 &= 0 & \text{D}\\
-x_3+x_5-3000+x_9 &= 0 & \text{E}\\
\end{align*}
Com isso adicionamos a coluna abaixo a matriz $A$ e resolvemos o sistema novamente.

In [54]:
A1 = hcat(A,[0; 0; 0; 0; 0; 0; -1; 1])
resolve(A1,c)

8-element Vector{Float64}:
  7000.0
  3500.0
  9000.0
 37000.0
 12000.0
 40500.0
 15000.0
 56000.0

Como estamos tratando apenas matrizes $A_{nxn}$, o resultado não é modificado, visto que a última coluna não é considerada pois $A1$ possui tamanho $8x9$.